In [1]:
from docplex.mp.model import Model  #导出库，只用这一个就够了
import matplotlib.pyplot as plt#选取了用户47的365天的数据
import numpy as np
import pandas as pd
import csv
import cplex
from cplex.exceptions import CplexError
import time
import math
import xlrd
import torch
from torch import nn
import os

In [2]:
#read file, for example
file_path = 'C:/Users/chenxihui/Desktop/code/project/2012.csv'
file = open(file_path)
data = []
for i in file.readlines():
    data.append(i)
file.close()

new_data = []
for row in data:
    tmp = row.strip('\n')  #去掉每行最后的回车符
    tmp = tmp.split(',')   #根据','来分割字符串，使之成为含有一个个数据的列表
    new_data.append(tmp)   #new_data的每一行数据就是一个列表
select_data = []

length=len(new_data)
for i in range(length):
    if(new_data[i][0]=='75'):#PVsize为8kwp
        select_data.append(new_data[i][3:])#[3:]

GC=[]
CL=[]
GG=[]
Battery=[]
Length_select_data = len(select_data)
for i in range(Length_select_data):
    if(select_data[i][0]=='GC'):
        GC.append(select_data[i][2:])
    elif(select_data[i][0]=='CL'):
        CL.append(select_data[i][2:])
    else:
        GG.append(select_data[i][2:])
        
Total_Load=[]
temp=[]
a=0.0
for i in range(Length_select_data):
    if(select_data[i][0]=='GC' and select_data[i+1][0]=='CL'):
        for j in range(len(select_data[i][2:])):
            a=pd.to_numeric(select_data[i][j+2])+pd.to_numeric(select_data[i+1][j+2])
            temp.append(a)
        Total_Load.append(temp)
    elif(select_data[i][0]=='GC' and select_data[i+1][0]!='CL'):
        Total_Load.append(select_data[i][2:])
    temp=[]
    
GC_temp=np.array(GC)
GC_array=[]
GC_array = GC_temp.astype(np.float)

GG_temp=np.array(GG)
GG_array=[]
GG_array = GG_temp.astype(np.float)*2

Total_Load_temp=np.array(Total_Load)
Total_Load_array=[]
Total_Load_array = Total_Load_temp.astype(np.float32)*2#功率
Total_Load_array[1]


array([2.488, 5.16 , 4.754, 1.53 , 0.176, 0.308, 0.19 , 0.194, 0.174,
       0.23 , 0.414, 0.88 , 1.802, 1.098, 0.394, 0.804, 0.246, 0.732,
       1.548, 3.568, 3.074, 0.738, 0.56 , 0.392, 0.33 , 0.502, 0.368,
       0.39 , 0.382, 0.348, 0.404, 0.33 , 0.326, 0.298, 0.406, 0.706,
       0.368, 0.424, 0.628, 0.594, 0.81 , 2.252, 2.708, 0.478, 1.596,
       1.588, 0.228, 0.666], dtype=float32)

In [3]:
pg_bar=np.max(Total_Load_array)
T_flat=0.235018
T_fit=0.09
T_tou=[0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,
       0.38588,0.38588, 0.38588,0.38588, 
      0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,
      0.38588,0.38588,0.38588,0.38588,0.38588,0.38588,
      0.37147,0.37147,0.37147,0.37147,
      0.2134,0.2134,0.2134,0.2134]

In [4]:
import openpyxl#之前将MILP得到的数据保存为xlsx文件，此为读取
def read_data(road):
    workbook=openpyxl.load_workbook("C:/Users/chenxihui/Desktop/code/project/processed_data/"+road+".xlsx")
    shenames=workbook.sheetnames
    worksheet=workbook.worksheets[0]
    name=worksheet.title 
    rows=worksheet.max_row
    columns=worksheet.max_column
    data_read=[[] for i in range(rows)]
    i=0
    for row in worksheet.rows:
        for cell in row:
            data_read[i].append(cell.value)
        i=i+1
    data_temp=np.array(data_read)
    data_array=[]
    data_array=data_temp.astype(np.float)
    return data_array

In [5]:
soc_y=read_data("soc")#从第1天开始

In [6]:
#此版本为362天的样本数据点集合,从第一天开始算，但是第一天里都是空集合
T_tou_temp=[]
for i in range(3):
    T_tou_temp.extend(T_tou)
days=365    
data_365=[[[] for _ in range(48)] for _ in range(days)]
for d in range(days):
    for i in range(48):
        data_temp=[]
        GG_temp=[]
        soc_temp=[]#前48个SOC（包含当前时刻）
    #print(len(soc_temp))
        #data_temp.extend(soc_temp)
        #print("data_temp：",data_temp)
        #print("#######################################")
        ToU_temp=[]
        load_temp=[]
        if(d<363 and d>0):
            if(i==0):
                soc_temp=np.hstack((soc_y[d-1][i+1:],soc_y[d][0:i+1]))
                ToU_temp=np.hstack((T_tou[i:],T_tou))
                #print("ToU_temp:",ToU_temp)
                load_temp=np.hstack((Total_Load_array[d][i:],Total_Load_array[d+1]))
                GG_temp=np.hstack((GG[d][i:],GG[d+1]))
            elif(i==47):
                soc_temp=soc_y[d]
                ToU_temp=np.hstack((T_tou,T_tou))
                #print("ToU_temp:",ToU_temp)
                load_temp=np.hstack((Total_Load_array[d],Total_Load_array[d+1]))
                GG_temp=np.hstack((GG[d],GG[d+1]))
            else:
                soc_temp=np.hstack((soc_y[d-1][i+1:],soc_y[d][0:i+1]))
                ToU_temp=np.hstack((T_tou[i:],T_tou,T_tou[:i]))
                GG_temp=np.hstack((GG[d][i:],GG[d+1],GG[d+2][:i]))
                load_temp=np.hstack((Total_Load_array[d][i:],Total_Load_array[d+1],Total_Load_array[d+2][:i]))
        #data_temp=np.hstack((data_temp,soc_temp))#用np.hstack
        data_temp=soc_temp
        #print(data_temp)
        data_temp=np.hstack((data_temp,ToU_temp))
        #print(data_temp)
        #print("#######################################")
        data_temp=np.hstack((data_temp,list(map(float, GG_temp))))
        #print(data_temp)
        #print("#######################################")
        data_temp=np.hstack((data_temp,load_temp))
        #print(len(data_temp[0][0]))
    #print(len(data_temp))
        data_365[d][i]=data_temp
        #print(data_365[d][i])
        #print("#######################################")

In [7]:
soc_label=[[]for i in range(301)]
for i in range(301):
    for j in np.arange(1,48):
        soc_label[i].append(soc_y[i+1][j])
    soc_label[i].append(soc_y[i+2][0])
soc_batch=np.split(soc_y[2:303],43,axis=0)
#np.array(soc_batch).shape  (43, 7, 48)

(43, 7, 48)

In [11]:
np.array(soc_batch).transpose((0,2,1)).shape 

(43, 48, 7)

In [9]:
a=np.array(data_365[1:302]).transpose((1,0,2))#[48, 301, 336]，
#第二天第一个时刻作为第一个输入样本
#np.array(soc_label).shape (301, 48)
data_batch=np.split(a,43,axis=1)
np.array(data_batch).shape

(43, 48, 7, 336)

In [24]:
torch.from_numpy(np.array(soc_batch).transpose((0,2,1))[0][:,:,np.newaxis]).float().shape

torch.Size([48, 7, 1])

In [1]:
# 超参设置7*42为294
TIME_STEP = 48  # RNN时间步长
INPUT_SIZE = 336  # RNN输入尺寸
INIT_LR = 10 # 初始学习率
N_EPOCHS = 100  # 训练回数
Batch_size=7
batch_nums=43

a=np.array(data_365[1:302]).transpose((1,0,2))#[48, 301, 336]，
#第二天第一个时刻作为第一个输入样本
#np.array(soc_label).shape (301, 48)
data_batch=np.split(a,43,axis=1)
#soc_batch=np.split(soc_y[2:303],43,axis=0)
#soc_batch=soc_y
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(
            input_size=INPUT_SIZE,
            hidden_size=20,  # RNN隐藏神经元个数20
            num_layers=1,  # RNN隐藏层个数
        )
        self.out = nn.Linear(20, 1).to(device)#全连接层，经过RNN网络后输出

    def forward(self, x, h):
        # x (time_step, batch_size, input_size)
        # h (n_layers, batch, hidden_size)
        # out (time_step, batch_size, hidden_size)
        out, h = self.rnn(x, h)
        #print("out:",out)
        #print("h:",h)
        prediction = self.out(out)
        #print("prediction:",prediction)
        return prediction, h


rnn = RNN()

optimizer = torch.optim.Adam(rnn.parameters(), lr=INIT_LR)
loss_func = nn.MSELoss()
#loss_func = torch.nn.CrossEntropyLoss()
h_state = None  # 初始化隐藏层
train_loss=[]
plt.figure()
plt.ion()
steps=[i for i in range(48*7)]
loss_min=20
for step in range(N_EPOCHS):
    error=0.0
    in_start = time.time()
    for batch_num in range(batch_nums):#此处分成了43个batch 
            #x = torch.from_numpy(np.array(a[batch_num])).float()  # 尺寸大小为(time_step, batch, input_size)
            x=torch.from_numpy(np.array(data_batch[batch_num])).float().to(device)
        #print(x.shape)
            #y = torch.from_numpy(np.array(soc_batch[batch_num]).transpose((1,0))[:,:,np.newaxis]).float()
            y=torch.from_numpy(np.array(soc_batch).transpose((0,2,1))[batch_num][:,:,np.newaxis]).float().to(device)
            #y = torch.from_numpy(np.array(soc_y[batch_num+2])[:,np.newaxis,np.newaxis]).float().to(device)#48*1*1
        #print("label:",y)

            prediction, h_state = rnn(x, h_state)  # RNN输出（预测结果，隐藏状态）
            h_state = h_state.detach()  # 这一行很重要，将每一次输出的中间状态传递下去(不带梯度)
            loss = loss_func(prediction, y)
            error += loss
            #print("epoch:",step,"batch_num:",batch_num,"loss:",loss)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if(loss_min>loss):#返回最小的loss时的
                loss_min=loss
                minloss_batch=batch_num
            #torch.save(rnn.state_dict(), 'C:/Users/chenxihui/Desktop/code/project/rnn.pt')
            if(loss<5):
                plt.cla()
                plt.plot(steps, np.array(soc_batch[batch_num]).flatten(), 'r-')
                plt.plot(steps, prediction.data.numpy().flatten(), 'b-')
                plt.draw()
                plt.pause(0.1)
    in_time = time.time() - in_start
    error /= batch_nums
    train_loss.append(error)
    print("Epoch: " + str(step) + ", Loss = " + str(error))
print("Finished. Time = " + str(time.time() - in_start))
print("save model...")
if not os.path.exists("./model/"):
    os.makedirs("./model/")
filepath = "./model/rnn2.model"
torch.save({"model_state_dict" : rnn.state_dict(),
            "optimizer_state_dict" : optimizer.state_dict()},
            filepath)   
print("Model has been saved.")
print("h:",h_state)
plt.ioff()
plt.show()


NameError: name 'np' is not defined

In [31]:
print("Testing...")
print("load model...")
filepath = "./model/rnn2.model"
checkpoint = torch.load(filepath)
rnn.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
rnn.eval()

test=np.array(data_365[302:342]).transpose((1,0,2))
test_loss=[]   
#hidden = rnn.init_hidden(1)
plt.figure()
plt.ion()
steps=[i for i in range(48*1)]
testloss_min=20
for step in range(40):
    test_x=torch.from_numpy(np.array(test[:,step,np.newaxis])).float().to(device)
    test_y = torch.from_numpy(np.array(soc_label[step+302])[:,np.newaxis,np.newaxis]).float().to(device)
    predict, h_state = rnn(test_x,h_state)
    #print(predict)    
    loss_test = loss_func(predict, test_y)
    test_loss.append(loss_test)
    if(testloss_min>loss_test):#返回最小的loss时的
        testloss_min=loss_test
        mintestloss_batch=step
    print("loss:",loss_test)
    if(loss<15):
        plt.cla()
        plt.plot(steps, np.array(soc_label[step]).flatten(), 'r-')
        plt.plot(steps, predict.data.numpy().flatten(), 'b-')
        plt.draw()
        plt.pause(0.1)
print("Test has been done!")
plt.ioff()
plt.show()

Testing...
load model...


IndexError: list index out of range

<Figure size 432x288 with 0 Axes>